In [ ]:
import numpy as np
import pandas as pd
import random

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import category_encoders as ce

from xgboost import XGBClassifier

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data Process

In [ ]:
data = pd.read_csv("../input/weather-dataset-rattle-package/weatherAUS.csv")
data[0:5].T

In [ ]:
n=len(data)
print(n)

In [ ]:
data1=data[data['Location']=='Albury']
data1[0:5].T

In [ ]:
data1.columns

In [ ]:
list_cols = [ 'Evaporation','WindDir9am', 'WindDir3pm',
       'Sunshine', 'WindGustDir', 'WindGustSpeed','Cloud9am', 'Cloud3pm', 
       'Temp9am', 'Temp3pm', 'RainToday', 'RainTomorrow']
ce_ohe = ce.OrdinalEncoder(cols=list_cols,handle_unknown='impute')
data2 = ce_ohe.fit_transform(data1)
data2[0:5].T


In [ ]:
data2.info()

In [ ]:
n=len(data2)
print(n)

In [ ]:
t=0
for item in data2['Date']:
    if item[0:4]=='2017':
        t+=1
print(t) 

In [ ]:
def dummy_date(df):
    df["year"] = df["Date"].apply(lambda x: x.split("-")[0])
    df["month"] = df["Date"].apply(lambda x: x.split("-")[1])
    df["day"] = df["Date"].apply(lambda x: x.split("-")[2])
    df.drop("Date",inplace=True,axis=1)
    return df

def LabelEncord_categorical(df):
    categorical_params = ["year","month","day"]
    for params in categorical_params:
        le = LabelEncoder()
        df[params] = le.fit_transform(df[params])
    return df

def dummies(df):
    categorical_params = ["year","month","day"]
    for params in categorical_params:
        dummies =  pd.get_dummies(df[params])
        df = pd.concat([df, dummies],axis=1)
    return df

def pre_processing(df):
    df = dummy_date(df)
    df = LabelEncord_categorical(df)
#    df = dummies(df)
    return df

data3 = pre_processing(data2)

In [ ]:
data3

In [ ]:
data3=data3.fillna(0)

In [ ]:
datay=data3['Rainfall'].apply(lambda x: x*10)
datax=data3.drop(['Rainfall','Location'],axis=1)

In [ ]:
pd.Series(datay).value_counts()

In [ ]:
datax=np.array(datax)
datay=np.array(datay)

In [ ]:
train=datax[0:t]
test=datax[t:]
trainy=datay[0:t]
testy=datay[t:]

In [ ]:
X = train
y = trainy

# Modeling

In [ ]:
clf = XGBClassifier(max_depth=3, n_estimators=1000, learning_rate=0.01)

In [ ]:
ss = ShuffleSplit(n_splits=5,train_size=0.8,test_size =0.2,random_state=0) 

for train_index, test_index in ss.split(X): 
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = y[train_index], y[test_index]
    clf.fit(X_train, Y_train) 
    print(clf.score(X_test, Y_test))

# Predict

In [ ]:
PRED = clf.predict(test)
PRED[0:20]

In [ ]:
ANS=testy
ANS[0:20]

In [ ]:
accuracy=accuracy_score(ANS,PRED)
print(accuracy)